In [3]:
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split

math_student_df = pd.read_csv('student-mat.csv', delimiter=';')
language_student_df = pd.read_csv('student-por.csv', delimiter=';')

#### Tasks:

##### Investigate impact of demographic factors on final grades
##### Compare across schools

In [4]:
math_GabPer_school = math_student_df[math_student_df['school'] == 'GP']
language_GabPer_school = language_student_df[language_student_df['school'] == 'GP']

math_Mous_school = math_student_df[math_student_df['school'] == 'MS']
language_Mous_school = language_student_df[language_student_df['school'] == 'MS']

In [23]:
# Drop non-demographic columns
demo_math_GabPer_school = math_GabPer_school.drop(['school', 'address', 'reason', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'absences', 'G1', 'G2', 'G3', 'romantic'], axis=1)
demo_language_GabPer_school = language_GabPer_school.drop(['school', 'address', 'reason', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'absences', 'G1', 'G2', 'G3', 'romantic'], axis=1)

demo_math_Mous_school = math_Mous_school.drop(['school', 'address', 'reason', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'absences', 'G1', 'G2', 'G3', 'romantic'], axis=1)
demo_language_Mous_school = language_Mous_school.drop(['school', 'address', 'reason', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'absences', 'G1', 'G2', 'G3', 'romantic'], axis=1)


# Change 'Medu', 'Fedu', and 'health' columns into strings for label encoding
convert_type = {'Medu': str, 'Fedu': str, 'health': str}

demo_math_GabPer_school = demo_math_GabPer_school.astype(convert_type)
demo_language_GabPer_school = demo_language_GabPer_school.astype(convert_type)
demo_math_Mous_school = demo_math_Mous_school.astype(convert_type)
demo_language_Mous_school = demo_language_Mous_school.astype(convert_type)

# Encode labels
demo_math_GabPer_school_dummies = pd.get_dummies(demo_math_GabPer_school, drop_first=True, dtype=int)
demo_language_GabPer_school_dummies = pd.get_dummies(demo_language_GabPer_school, drop_first=True)
demo_math_Mous_school_dummies = pd.get_dummies(demo_math_Mous_school, drop_first=True)
demo_language_Mous_school_dummies = pd.get_dummies(demo_language_Mous_school, drop_first=True)

# G3 Response Variable
G3_math_gab = math_GabPer_school['G3']
G3_lang_gab = language_GabPer_school['G3']
G3_math_mous = math_Mous_school['G3']
G3_lang_mous = language_Mous_school['G3']

In [25]:
# Reduce number of features using Recursive Feature Elimination (RFE) and fit linear regression for math class at Gabriel Pereira

model = LinearRegression()
rfe = RFE(estimator=model, n_features_to_select=2)
rfe = rfe.fit(demo_math_GabPer_school_dummies, G3_math_gab)

# Get the selected features
selected_features = demo_math_GabPer_school_dummies.columns[rfe.support_]
print("Selected features:", selected_features)

# Fit the model again with selected features
X_selected = demo_math_GabPer_school_dummies[selected_features]
X_selected = sm.add_constant(X_selected)
model_selected = sm.OLS(G3_math_gab, X_selected).fit()
print(model_selected.summary())

Selected features: Index(['Fedu_1', 'Fedu_2'], dtype='object')
                            OLS Regression Results                            
Dep. Variable:                     G3   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     3.190
Date:                Tue, 18 Jun 2024   Prob (F-statistic):             0.0424
Time:                        17:46:48   Log-Likelihood:                -1026.0
No. Observations:                 349   AIC:                             2058.
Df Residuals:                     346   BIC:                             2070.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [27]:
# Reduce number of features using Recursive Feature Elimination (RFE) and fit linear regression for langauge class at Gabriel Pereira

model = LinearRegression()
rfe = RFE(estimator=model, n_features_to_select=2)
rfe = rfe.fit(demo_language_GabPer_school_dummies, G3_lang_gab)

# Get the selected features
selected_features = demo_language_GabPer_school_dummies.columns[rfe.support_]
print("Selected features:", selected_features)

# Fit the model again with selected features
X_selected = demo_language_GabPer_school_dummies[selected_features]
X_selected = sm.add_constant(X_selected)
model_selected = sm.OLS(G3_lang_gab, X_selected.astype(int)).fit()
print(model_selected.summary())

Selected features: Index(['sex_M', 'Medu_4'], dtype='object')
                            OLS Regression Results                            
Dep. Variable:                     G3   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     20.92
Date:                Tue, 18 Jun 2024   Prob (F-statistic):           2.18e-09
Time:                        17:47:20   Log-Likelihood:                -987.96
No. Observations:                 423   AIC:                             1982.
Df Residuals:                     420   BIC:                             1994.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------

In [36]:
# Reduce number of features using Recursive Feature Elimination (RFE) and fit linear regression for math class at Mous

model = LinearRegression()
rfe = RFE(estimator=model, n_features_to_select=2)
rfe = rfe.fit(demo_math_Mous_school_dummies, G3_math_mous)

# Get the selected features
selected_features = demo_math_Mous_school_dummies.columns[rfe.support_]
print("Selected features:", selected_features)

# Fit the model again with selected features
X_selected = demo_math_Mous_school_dummies[selected_features]
X_selected = sm.add_constant(X_selected)
model_selected = sm.OLS(G3_math_mous, X_selected.astype(int)).fit()
print(model_selected.summary())

Selected features: Index(['Fedu_4', 'Mjob_teacher'], dtype='object')
                            OLS Regression Results                            
Dep. Variable:                     G3   R-squared:                       0.138
Model:                            OLS   Adj. R-squared:                  0.098
Method:                 Least Squares   F-statistic:                     3.442
Date:                Tue, 18 Jun 2024   Prob (F-statistic):             0.0411
Time:                        18:14:09   Log-Likelihood:                -127.77
No. Observations:                  46   AIC:                             261.5
Df Residuals:                      43   BIC:                             267.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------

In [37]:
# Reduce number of features using Recursive Feature Elimination (RFE) and fit linear regression for language class at Mous

model = LinearRegression()
rfe = RFE(estimator=model, n_features_to_select=2)
rfe = rfe.fit(demo_language_Mous_school_dummies, G3_lang_mous)

# Get the selected features
selected_features = demo_language_Mous_school_dummies.columns[rfe.support_]
print("Selected features:", selected_features)

# Fit the model again with selected features
X_selected = demo_language_Mous_school_dummies[selected_features]
X_selected = sm.add_constant(X_selected)
model_selected = sm.OLS(G3_lang_mous, X_selected.astype(int)).fit()
print(model_selected.summary())

Selected features: Index(['famsize_LE3', 'Fjob_teacher'], dtype='object')
                            OLS Regression Results                            
Dep. Variable:                     G3   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     5.188
Date:                Tue, 18 Jun 2024   Prob (F-statistic):            0.00628
Time:                        18:17:21   Log-Likelihood:                -618.76
No. Observations:                 226   AIC:                             1244.
Df Residuals:                     223   BIC:                             1254.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------